In [1]:
# Main import block
import random
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup as bs
import numpy as np
from datetime import date
import datetime

import time
from tqdm import tqdm

# Upload to google sheets
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

import signal
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')


In [5]:
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date

In [2]:
def shmet_parser():
    def shmet_day():
        day = date.today()
        today = day.strftime('%Y-%m-%d')
        url = f'https://en.shmet.com/api/rest/enweb/spot/getChartPrices?scId=811&startDate={day}&endDate={day}&_'#=1690186841910'

        response = requests.get(url)
        if response.status_code == 200:  # Проверка успешности запроса
            data = response.json()  # Получение JSON-данных из ответа

        with open("./data/shmet.json", "w") as file:  # Открытие файла для записи
            json.dump(data, file)  # Запись JSON-данных в файл

        temp = pd.read_json('./data/shmet.json')['data']['spotPrices']
        data = pd.DataFrame(temp)

        data['dateTime'] = data['dateTime'].apply(date_format)
        data.rename({'dateTime': 'date'}, axis=1, inplace=True)
        data['date'] = pd.to_datetime(data['date']).dt.date
        data['date'] = pd.to_datetime(data['date'])

        currency_df = pd.read_excel(
            '../parser_beta/data/cb_curr.xlsx', index_col=0)

        temp_cur = currency_df.query("Id == 'USD' or Id == 'China_Yuan'")

        temp_cur['Value'] = temp_cur['Value']/temp_cur['Nominal']

        temp_cur = temp_cur[temp_cur['Date'] ==
                            today].reset_index().drop(['index', 'Nominal'], axis=1)

        pivot_table = temp_cur.pivot(
            index='Date', columns='Id', values='Value')

        pivot_table = pivot_table.reset_index().rename(
            columns={'Date': 'date', 'China_Yuan': 'Yuan'})

        shmet_raw = data.merge(
            right=pivot_table, right_on='date', left_on='date')
        shmet_raw['price'] = pd.to_numeric(shmet_raw['price'])
        shmet_raw['rate'] = round(shmet_raw['Yuan'] / shmet_raw['USD'], 3)
        shmet_raw['price_usd'] = round(
            shmet_raw['price'] * shmet_raw['rate'], 3)
        shmet_raw = shmet_raw.reindex(
            columns=['date', 'price', 'unit', 'USD', 'Yuan', 'price_usd', 'rate'])

        return shmet_raw

    df = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    df.drop_duplicates(inplace=True)

    try:
        day_row = shmet_day()
        shmet_db = pd.concat([day_row, df], axis=0)

        shmet_db = shmet_db.reset_index(drop=True)
        shmet_db.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
                "../parser_beta/data/shmet_historical.xlsx",
                date_format="YYYY-MM-DD",
                datetime_format="YYYY-MM-DD") as writer:
            shmet_db.to_excel(writer, sheet_name='SHMET')

        google_upload(final, 'SHMET')

    except (IndexError, KeyError) as e:
        print('Shmet day is empty, please check the source.')
        return None

    final = pd.read_excel('./data/shmet_historical.xlsx', index_col=0)
    final.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
            ate_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final.to_excel(writer, sheet_name='SHMET')

    return final

In [6]:
def shmet_day():
    day = date.today()
    today = day.strftime('%Y-%m-%d')
    # =1690186841910'
    url = f'https://en.shmet.com/api/rest/enweb/spot/getChartPrices?scId=811&startDate={day}&endDate={day}&_'

    response = requests.get(url)
    if response.status_code == 200:  # Проверка успешности запроса
        data = response.json()  # Получение JSON-данных из ответа

    with open("./data/shmet.json", "w") as file:  # Открытие файла для записи
        json.dump(data, file)  # Запись JSON-данных в файл

    temp = pd.read_json('./data/shmet.json')['data']['spotPrices']
    data = pd.DataFrame(temp)

    data['dateTime'] = data['dateTime'].apply(date_format)
    data.rename({'dateTime': 'date'}, axis=1, inplace=True)
    data['date'] = pd.to_datetime(data['date']).dt.date
    data['date'] = pd.to_datetime(data['date'])

    return data


In [7]:
shmet_day()

,date,price,unit
0,2023-08-01,70925,Yuan/MT
